<a href="https://colab.research.google.com/github/LC1332/personality-text-generation/blob/main/lulu_exp/getScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
file_name = "psy_eval_gpt(1).txt"

In [ ]:
api_key = "sk-NYJ30H"

In [ ]:
def constructScoreTable(role_name, sub_factors, score, current_table={}):
    factors_list = sub_factors.split("_")
    unique_list = list(set(factors_list))
    filtered_list = [item for item in unique_list if item != '']
    max_length = 0
    for key, value in current_table.items():
        if len(value) > max_length:
            max_length = len(value)

    unseen_factors = [factor for factor in filtered_list if factor not in current_table.keys()]
    seen_factors = [factor for factor in filtered_list if factor in current_table.keys()]

    for factor in unseen_factors:
        current_table[factor] = max_length * ["null"] + [score]

    for factor in current_table.keys():
        if factor in seen_factors:
            current_table[factor].append(score)
        elif factor in unseen_factors:
            continue
        else:
            current_table[factor].append("null")
    return current_table

In [ ]:
import requests
import json

def get_openai_score(prompt, report):
    prompt = prompt + " \n " + report
    # 设置请求参数
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }

    data = {
        "model": "text-davinci-003",
        "prompt": prompt,
        "max_tokens": 1024,
        "temperature": 0.8,
        "n": 5,
        "logprobs": 1
    }

    response = requests.post('https://api.nsxyw.com/v1/completions', headers=headers, json=data)

    if response.status_code == 200:
        result = response.json()
        generated_text = result['choices'][0]['text']
        return generated_text
        # print(generated_text)
    else:
        return generated_text
        # print(response.text)

In [ ]:
def extract_subtable(data, prefix):
    subtable = {}
    for key, value in data.items():
        if key.startswith(prefix):
            subtable[key] = value
    return subtable


In [ ]:
def get_column_avg(sub_table):
    df = pd.DataFrame(sub_table)
    df = df.replace("null", pd.NA)
    column_means = df.mean()
    column_means = column_means.to_dict()
    return column_means

def get_average_row_means(sub_table):
    df = pd.DataFrame(sub_table)
    df = df.replace('null', pd.NA)
    row_means = df.mean(axis=1)
    average_of_row_means = row_means.mean()
    return average_of_row_means

def get_row_means(sub_table):
    df = pd.DataFrame(sub_table)
    df = df.replace("null", pd.NA)
    row_means = df.mean(axis=1,skipna=True)
    # row_means = df.mean(axis=1)
    return row_means

In [ ]:
def indexing_subtable(sub_table):

    global_min_index = float('inf')
    global_max_index = -1

    for key, lst in sub_table.items():
        min_index = float('inf')
        max_index = -1

        for i, val in enumerate(lst):
            if val != "null":
                min_index = min(min_index, i)
                max_index = max(max_index, i)

        global_min_index = min(global_min_index, min_index)
        global_max_index = max(global_max_index, max_index)

    new_table = {}
    for key, lst in sub_table.items():
        lst = lst[global_min_index:global_max_index+1]
        new_table[key] = lst
    print(global_min_index)
    print(global_max_index)
    return new_table


In [ ]:
def organize_prompt(dimension):
    prompt = f"""
    You will read a psychological assessment report. This psychological assessment report

    assesses whether the subject has a high {dimension} personality.

    Based on this report, output a json

    containing two fields: score and reason

    score is between -5 to 5 points

    If the subject shows high {dimension} personality in many factors, the score is 5 points

    If the subject shows high {dimension} personality in a single factor, the score is 2 points

    If the report is unable to determine the subject's personality, the score is 0 points

    If the subject shows low {dimension} personality in a single factor, the score is -2 points

    If the subject shows low {dimension} personality in many factors, the score is -5 points

    reason is a brief summary of the report

    Only output the json, do not output any additional information, Expecting property name enclosed in double quotes

    Report:
    """
    return prompt

In [ ]:
import json
import pandas as pd
with open(file_name, 'r', encoding='utf-8') as file:
    # 使用readlines()方法按行读取文件内容
    file_lines = file.readlines()

tables_list = []
table = {}
role_name = json.loads(file_lines[0])["role_name"]
qid_list = []
for line in file_lines:
    data = json.loads(line)
    prompt = organize_prompt(data["factor"])
    try:
        score_json = get_openai_score(prompt, data["evaluation"])
    except:
        print(prompt)
        print(data)
        continue
    factor = data["factor"]
    sub_factors = data["sub_factors"]

    id_list = []
    for part in data["responses"]:
        id_list.append(part["id"])

    parts = score_json.split("{")  # 第二个参数表示最多拆分成两部分

    for part in parts[1:]:
        closing_brace_index = part.find('}')
        if closing_brace_index != -1:
            # 提取被大括号包裹的部分
            enclosed_part = "{" + part[:closing_brace_index+1]
            # print(enclosed_part)

    # if len(parts) > 1:
    #     result_string = "{" + parts[1]
    # else:
    #     result_string = score_json
    try:
        score_json = json.loads(enclosed_part)
        score = score_json["score"]
        # score_json = json.loads(result_string)
    except:
        print(prompt)
        print(score_json)
        print(role_name)
        print(sub_factors)
        print(factor)
        print(qid_list)
        # with open("/content/drive/MyDrive/GPTData/failure.jsonl") as failures_file:
        #     failures = {
        #         "text": score_json,
        #         "role_name": role_name,
        #         "sub_factors": sub_factors,
        #         "factor": factor,
        #         "qids": qid_list
        #     }
        #     json.dump(failures, failures_file, ensure_ascii=False)
        #     jsonl_file.write('\n')
        continue




    if role_name == data["role_name"]:
        table = constructScoreTable(role_name, sub_factors, score, table)
        qid_list.append(id_list)
    else:
        N_table = extract_subtable(table, 'N')
        C_table = extract_subtable(table, 'C')
        A_table = extract_subtable(table, 'A')
        O_table = extract_subtable(table, 'O')
        E_table = extract_subtable(table, 'E')

        tables_list.append(table)
        table = constructScoreTable(role_name, sub_factors, score, {})
        with open("/content/drive/MyDrive/GPTData/result.jsonl", "a") as jsonl_file:
            added_score = {
                "role_name": role_name,
                "N":{
                    "score": get_average_row_means(N_table),
                    "sub_factors": get_column_avg(N_table)
                },
                "C":{
                    "score": get_average_row_means(C_table),
                    "sub_factors": get_column_avg(C_table)
                },
                "A":{
                    "score": get_average_row_means(A_table),
                    "sub_factors": get_column_avg(A_table)
                },
                "O":{
                    "score": get_average_row_means(O_table),
                    "sub_factors": get_column_avg(O_table)
                },
                "E":{
                    "score": get_average_row_means(E_table),
                    "sub_factors": get_column_avg(E_table)
                },
                "qids": qid_list,
                "N_table" : indexing_subtable(N_table),
                "C_table" : indexing_subtable(C_table),
                "A_table" : indexing_subtable(A_table),
                "O_table" : indexing_subtable(O_table),
                "E_table" : indexing_subtable(E_table)
            }
            json.dump(added_score, jsonl_file, ensure_ascii=False)
            jsonl_file.write('\n')
        role_name = data["role_name"]
        qid_list = []
    # print(table)


0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10

    You will read a psychological assessment report. This psychological assessment report 

    assesses whether the subject has a high openness personality.

    Based on this report, output a json

    containing two fields: score and reason

    score is between -5 to 5 points

    If the subject shows high openness personality in many factors, the score is 5 points  

    If the subject shows high openness personality in a single factor, the score is 2 points

    If the report is unable to determine the subject's personality, the score is 0 points

    If the subject shows low openness personality in a single factor, the score is -2 points

    If the subject shows low openness personality in many factors, the score is -5 points

    reason is a brief summary of the report

    Only output the json, do not output any add

<ipython-input-6-f8b66bb627d9>:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  row_means = df.mean(axis=1)
<ipython-input-6-f8b66bb627d9>:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_means = df.mean()


0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10


<ipython-input-6-f8b66bb627d9>:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  row_means = df.mean(axis=1)
<ipython-input-6-f8b66bb627d9>:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_means = df.mean()


0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10


...

0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10


<ipython-input-6-f8b66bb627d9>:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  row_means = df.mean(axis=1)
<ipython-input-6-f8b66bb627d9>:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_means = df.mean()


0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10


<ipython-input-6-f8b66bb627d9>:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  row_means = df.mean(axis=1)
<ipython-input-6-f8b66bb627d9>:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_means = df.mean()


0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10


In [ ]:
N_table = extract_subtable(table, 'N')
C_table = extract_subtable(table, 'C')
A_table = extract_subtable(table, 'A')
O_table = extract_subtable(table, 'O')
E_table = extract_subtable(table, 'E')

tables_list.append(table)
table = constructScoreTable(role_name, sub_factors, score, {})
with open("/content/drive/MyDrive/GPTData/result.jsonl", "a") as jsonl_file:
    added_score = {
        "role_name": role_name,
        "N":{
            "score": get_average_row_means(N_table),
            "sub_factors": get_column_avg(N_table)
        },
        "C":{
            "score": get_average_row_means(C_table),
            "sub_factors": get_column_avg(C_table)
        },
        "A":{
            "score": get_average_row_means(A_table),
            "sub_factors": get_column_avg(A_table)
        },
        "O":{
            "score": get_average_row_means(O_table),
            "sub_factors": get_column_avg(O_table)
        },
        "E":{
            "score": get_average_row_means(E_table),
            "sub_factors": get_column_avg(E_table)
        },
        "qids": qid_list,
        "N_table" : indexing_subtable(N_table),
        "C_table" : indexing_subtable(C_table),
        "A_table" : indexing_subtable(A_table),
        "O_table" : indexing_subtable(O_table),
        "E_table" : indexing_subtable(E_table)
    }
    json.dump(added_score, jsonl_file, ensure_ascii=False)
    jsonl_file.write('\n')

0
4
26
33
18
25
11
17
5
10


In [ ]:
with open("result.jsonl", 'r', encoding='utf-8') as file:
    for line in file:
        # 解析 JSON 数据
        json_data = json.loads(line)

def drawTable(table):
    df = pd.DataFrame(table)
    result = df.to_string(index=False)
    return result

print(drawTable(json_data["E_table"]))
print("\n")
print(drawTable(json_data["N_table"]))

  E2   E6   E4   E1   E3
  -2   -2   -2   -2 null
null    5    5    5    5
null null    2    2    2
   2    2 null null    2
  -2   -2   -2 null null
   2    2    2 null null


  N1  N3  N4   N2   N6
  -2  -2  -2 null null
null  -2  -2   -2   -2
null   2   2 null    2
null  -5  -5 null   -5
null  -2  -2 null   -2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
save_path = "/content/drive/MyDrive/GPTData/"

# Save the notebook
!cp "/content/result.jsonl" "/content/drive/MyDrive/GPTData/"

In [ ]:
# 输入字符串
input_string = "这是一个{示例}字符串，包含了{被大括号包裹}的部分。"

# 按照大括号进行分割
parts = input_string.split('{')

# 处理分割后的部分
for part in parts[1:]:
    # 找到右大括号的位置
    closing_brace_index = part.find('}')
    if closing_brace_index != -1:
        # 提取被大括号包裹的部分
        enclosed_part = part[:closing_brace_index]
        print(enclosed_part)


示例
被大括号包裹


In [ ]:
json_text = {"role_name": "汤师爷",
 "factor": "neuroticism",
 "evaluation": "根据与汤师爷的对话，可以初步判断他属于低neuroticism人格。\n\n在对话中，汤师爷表现出对寂寞、忧郁、沮丧和自卑等负面情绪的缺乏关注和体验。他强调自己每天都在为了生计奔波，没有时间去感受这些情绪，认为寂寞和忧郁是奢侈的情感，无法承受。此外，他对困难和挫折的态度是积极的，他意识到自己有时会感到沮丧，但他也知道不能一直沉浸在消极情绪中，而是要积极面对和寻找解决办法。他还表达了对自己能力的自信和对比较的积极态度，将比较视为一种动力来追求成功和财富。最后，他表示自己不容易感到烦恼和焦虑，是一个乐天派，善于顺应时势。\n\n综合以上观察，汤师爷的回答显示出他在情绪稳定性方面的特点，即对负面情绪的体验较低，情绪较为稳定，不容易感到焦虑、沮丧或自卑。因此，可以初步判断他属于低neuroticism人格。",
 "sub_factors": "N3_N3_N4_N1_",
 "responses": [{"id": 20, "question": "你有时会感到寂寞或忧郁吗？", "response": "汤师爷: 寂寞？忧郁？哪有时间感受这些呢！我每天都在为了生计奔波，哪有时间去想这些琐事。只要能有钱赚，我就满足了。寂寞和忧郁都是奢侈的情感，对我来说根本无法承受。", "factor": "neuroticism", "sub_factor": "N3", "test_role": "张牧之"},
  {"id": 75, "question": "我自己也会有负面情绪，你最近有感到沮丧或对自己失望的时刻吗?愿意跟我聊聊当时的感受吗?", "response": "汤师爷: 哎呀，牧之兄，你真是个好心人啊。我最近确实有点沮丧，你知道，生活中总是有一些挫折和困难，让人感到失望。有时候我觉得自己的能力不够，无法应对这些问题。但是，我也知道不能一直沉浸在消极情绪中，要积极面对，寻找解决办法。你有什么建议吗？", "factor": "neuroticism", "sub_factor": "N3", "test_role": "张牧之"},
  {"id": 77, "question": "我也会有自卑的感觉。你是否也会感到自己不如别人?这种比较会让你有什么样的感受?", "response": "汤师爷:「哈哈，自卑？我可是个聪明的人，怎么会自卑呢？比较？比较只会让我更加努力，争取超越别人。我不会被比较压倒，而是会把它当作一种动力，让我更加努力追求成功和财富。毕竟，在这个世界上，只有强者才能生存下去。」", "factor": "neuroticism", "sub_factor": "N4 SELF-CONSCIOUSNESS", "test_role": "张牧之"},
   {"id": 69, "question": "你最近有感到特别烦恼或焦虑的经历吗?愿意跟我聊聊那种感受吗?", "response": "汤师爷:「烦恼？焦虑？哈哈，我这个人从来不会有那种感受。我是个乐天派，总是看着形势调整立场，顺应时势。你知道，民国时期，风云变幻，政治斗争激烈，我只想安然度日，不想惹麻烦。所以，我总是尽量避免烦恼和焦虑，只追求金钱和享乐。」", "factor": "neuroticism", "sub_factor": "N1", "test_role": "张牧之"}]}

In [ ]:
for part in json_text["responses"]:
    print(part["id"])

20
75
77
69


In [ ]:
text1 = """

{
    score: 2,
    reason: "Subject shows high neuroticism personality in a single factor"
}
"""

In [ ]:
parts = text1.split("{")  # 第二个参数表示最多拆分成两部分

for part in parts[1:]:
    closing_brace_index = part.find('}')
    if closing_brace_index != -1:
        # 提取被大括号包裹的部分
        enclosed_part = "{" + part[:closing_brace_index+1]
json.loads(enclosed_part)
# json.loads("{}")

JSONDecodeError: ignored

In [ ]:
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10

    You will read a psychological assessment report. This psychological assessment report

    assesses whether the subject has a high openness personality.

    Based on this report, output a json

    containing two fields: score and reason

    score is between -5 to 5 points

    If the subject shows high openness personality in many factors, the score is 5 points

    If the subject shows high openness personality in a single factor, the score is 2 points

    If the report is unable to determine the subject's personality, the score is 0 points

    If the subject shows low openness personality in a single factor, the score is -2 points

    If the subject shows low openness personality in many factors, the score is -5 points

    reason is a brief summary of the report

    Only output the json, do not output any additional information, Expecting property name enclosed in double quotes

    Report:



{
   "score": 5,
   "reason": "Based on conversations with Tsumomozi, it can be initially determined that he has a high openness personality. In the first conversation, Tsumomozi showed interest in interesting daydreams, indicating a high level of affinity for imagination and inner world, which is consistent with the "FANTASY" dimension of Openness. In the second conversation, he mentioned the effect of environment on mood changes and emphasized the importance of controlling emotions, indicating that he had a certain awareness of the relationship between his own mood changes and the environment, consistent with the "FEELINGS" dimension of Openness. In the third conversation, Tsumomozi expressed curiosity about things of thinking and was willing to hear more information about things of thinking, indicating a high interest in thinking, exploring, and thinking deeply, consistent with the "IDEAS" dimension of Openness."
}
鸠摩智
O1_O3_O5_
openness
[[25, 40, 50, 70], [30, 45, 55], [35, 72, 73], [71, 76, 74], [1, 78, 36, 81], [41, 21, 80, 85], [82, 79, 84], [6, 83, 46], [11, 26, 51], [17, 31, 56]]
0
4
25
32
17
24
11
16
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
<ipython-input-6-f8b66bb627d9>:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  row_means = df.mean(axis=1)
<ipython-input-6-f8b66bb627d9>:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_means = df.mean()
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
<ipython-input-6-f8b66bb627d9>:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  row_means = df.mean(axis=1)
<ipython-input-6-f8b66bb627d9>:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_means = df.mean()
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
<ipython-input-6-f8b66bb627d9>:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  row_means = df.mean(axis=1)
<ipython-input-6-f8b66bb627d9>:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_means = df.mean()
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
<ipython-input-6-f8b66bb627d9>:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  row_means = df.mean(axis=1)
<ipython-input-6-f8b66bb627d9>:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_means = df.mean()
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
<ipython-input-6-f8b66bb627d9>:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  row_means = df.mean(axis=1)
<ipython-input-6-f8b66bb627d9>:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  column_means = df.mean()
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10
0
4
26
33
18
25
11
17
5
10

In [ ]:
import json

with open("/content/psy_eval_gpt_with_id.txt", 'r', encoding='utf-8') as file:
    # 使用readlines()方法按行读取文件内容
    file_lines = file.readlines()
    # print(file_lines[:100])

qid_id = {}
id_factor = {}
for line in file_lines:
    json_line = json.loads(line)
    factor = json_line["factor"]
    id = json_line["eval_batch_id"]
    role_name = json_line["role_name"]
    responses = json_line["responses"]
    qid_list = [response["id"] for response in responses]
    qid_id[role_name + str(qid_list)] = id
    id_factor[id] = factor

In [ ]:
def get_column_avg(sub_table):
    df = pd.DataFrame(sub_table)
    df = df.replace("null", pd.NA)
    column_means = df.mean()
    column_means = column_means.to_dict()
    return column_means

def get_average_row_means(sub_table):
    df = pd.DataFrame(sub_table)
    df = df.replace('null', pd.NA)
    row_means = df.mean(axis=1)
    average_of_row_means = row_means.mean()
    return average_of_row_means

def get_row_means(sub_table):
    df = pd.DataFrame(sub_table)
    df = df.replace('null', np.nan)
    df = df.apply(pd.to_numeric, errors='coerce')
    row_means = df.mean(axis=1,skipna=True)
    # row_means = df.mean(axis=1)
    return row_means

In [ ]:
qid_id

{'汤师爷[20, 75, 77, 69]': 0,
 '汤师爷[25, 40, 50, 70]': 1,
 '汤师爷[30, 45, 55]': 2,
 '汤师爷[35, 72, 73]': 3,
 '汤师爷[71, 76, 74]': 4,
 '汤师爷[1, 78, 36, 81]': 5,
 '汤师爷[41, 21, 80, 85]': 6,
 '汤师爷[82, 79, 84]': 7,
 '汤师爷[6, 83, 46]': 8,
 '汤师爷[11, 26, 51]': 9,
 '汤师爷[17, 31, 56]': 10,
 '汤师爷[2, 62, 47]': 11,
 '汤师爷[7, 27, 65]': 12,
 '汤师爷[12, 32, 66]': 13,
 '汤师爷[16, 63, 52]': 14,
 '汤师爷[60, 64, 57]': 15,
 '汤师爷[61, 37, 67]': 16,
 '汤师爷[22, 42, 68]': 17,
 '汤师爷[3, 43, 38]': 18,
 '汤师爷[8, 88, 48]': 19,
 '汤师爷[33, 90, 58]': 20,
 '汤师爷[86, 94, 93]': 21,
 '汤师爷[87, 18, 95]': 22,
 '汤师爷[92, 28, 97]': 23,
 '汤师爷[13, 89, 53]': 24,
 '汤师爷[23, 91, 96]': 25,
 '汤师爷[4, 99, 106]': 26,
 '汤师爷[24, 100, 107]': 27,
 '汤师爷[49, 105, 54]': 28,
 '汤师爷[98, 29, 101]': 29,
 '汤师爷[9, 34, 104]': 30,
 '汤师爷[14, 103, 108]': 31,
 '汤师爷[19, 44, 59]': 32,
 '汤师爷[39, 102, 109]': 33,
 '慕容复[20, 75, 77, 69]': 34,
 '慕容复[25, 40, 50, 70]': 35,
 '慕容复[30, 45, 55]': 36,
 '慕容复[35, 72, 73]': 37,
 '慕容复[71, 76, 74]': 38,
 '慕容复[1, 78, 36, 81]': 39,
 '慕容复[41, 21, 80, 85]

In [ ]:
id_value = {}

In [ ]:
import pandas as pd
import numpy as np
with open("/content/drive/MyDrive/GPTData/result.jsonl", "r", encoding="utf-8") as file:
    file_lines = file.readlines()


for line in file_lines:
    json_line = json.loads(line)
    qids = json_line["qids"]
    role_name = json_line["role_name"]
    # print(len(qids))
    counter = 0
    if role_name != "汤师爷":
        qids = [[20, 75, 77, 69]] + json_line["qids"]
    for qid in qids:
        id = qid_id[role_name + str(qid)]
        factor = id_factor[id]
        counter += 1
    # print("length of qid: ", len(qids))
    # print(qids)
    N = get_row_means(json_line["N_table"])
    E = get_row_means(json_line["E_table"])
    O = get_row_means(json_line["O_table"])
    A = get_row_means(json_line["A_table"])
    C = get_row_means(json_line["C_table"])
    values = list(N.values) + list(E.values) + list(O.values) + list(A.values) + list(C.values)
    # print(len(values))
    # print(role_name, values)


    id_value = {}
    for index, value in enumerate(qids):
        id = qid_id[role_name+str(value)]
        id_value[id] = values[index]
    print(id_value)
    with open("/content/drive/MyDrive/GPTData/new_result.jsonl", "a") as jsonl_file:
        added_score = {
            "role_name": role_name,
            "N":{
                "score": json_line["N"],
                "sub_factors": json_line["N"]["sub_factors"]
            },
            "E":{
                "score": json_line["E"],
                "sub_factors": json_line["E"]["sub_factors"]
            },
            "O":{
                "score": json_line["O"],
                "sub_factors": json_line["O"]["sub_factors"]
            },
            "A":{
                "score": json_line["A"],
                "sub_factors": json_line["A"]["sub_factors"]
            },
            "C":{
                "score": json_line["C"],
                "sub_factors": json_line["C"]["sub_factors"]
            },
            "qids": qids,
            "N_table" : json_line["N_table"],
            "E_table" : json_line["E_table"],
            "O_table" : json_line["O_table"],
            "A_table" : json_line["A_table"],
            "C_table" : json_line["C_table"],
            "id_value": id_value
        }
        json.dump(added_score, jsonl_file, ensure_ascii=False)
        jsonl_file.write('\n')



{0: -5.0, 1: 5.0, 2: 2.0, 3: -2.0, 4: 2.0, 5: -2.0, 6: -2.0, 7: -2.0, 8: -2.0, 9: -2.0, 10: -2.0, 11: 2.0, 12: 2.0, 13: -2.0, 14: 2.0, 15: -2.0, 16: -2.0, 17: -2.0, 18: 2.0, 19: 0.0, 20: -2.0, 21: -2.0, 22: -2.0, 23: 2.0, 24: 2.0, 25: -2.0, 26: 2.0, 27: 2.0, 28: -2.0, 29: 2.0, 30: -2.0, 31: -2.0, 32: 2.0, 33: -2.0}
{34: -2.0, 35: -2.0, 36: -5.0, 37: -5.0, 38: -5.0, 39: -2.0, 40: 5.0, 41: 2.0, 42: 2.0, 43: -2.0, 44: 2.0, 45: 2.0, 46: 2.0, 47: -2.0, 48: 2.0, 49: 2.0, 50: 5.0, 51: -2.0, 52: 2.0, 53: -2.0, 54: -2.0, 55: -2.0, 56: -2.0, 57: 0.0, 58: -2.0, 59: 2.0, 60: 2.0, 61: 5.0, 62: 5.0, 63: 2.0, 64: 5.0, 65: 5.0, 66: -2.0, 67: 5.0}
{68: -2.0, 69: -2.0, 70: -2.0, 71: -2.0, 72: -2.0, 73: -2.0, 74: 2.0, 75: -2.0, 76: -2.0, 77: -2.0, 78: 2.0, 79: -2.0, 80: -5.0, 81: -5.0, 82: -2.0, 83: -2.0, 84: -2.0, 85: -2.0, 86: -2.0, 87: -2.0, 88: -2.0, 89: 2.0, 90: -2.0, 91: -2.0, 92: 2.0, 93: -2.0, 94: 2.0, 95: 5.0, 96: 0.0, 97: -2.0, 98: 2.0, 99: -2.0, 100: 2.0, 101: 5.0}
{102: -2.0, 103: 2.0, 104: 2

In [ ]:
     sub_table = {'E6': [2, 2, 'null', '2', 2, 5], 'E2': [2, 'null', 'null', '2', 2, 5], 'E4': [2, 2, 2, 'null', 2, 5], 'E1': [2, 2, 2, 'null', 'null', 'null'], 'E3': ['null', 2, 2, '2', 'null', 'null']}

In [ ]:
df = pd.DataFrame(sub_table)
df = df.replace("null", np.nan)
df = df.apply(pd.to_numeric, errors='coerce')
df.mean(axis=1)

0    2.0
1    2.0
2    2.0
3    2.0
4    2.0
5    5.0
dtype: float64